<a href="https://colab.research.google.com/github/CoGian/greek-bioBERT/blob/main/Finetune_greekBERT_on_biomedical_data_with_MLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 1.5MB 5.3MB/s 
     |████████████████████████████████| 890kB 46.1MB/s 
     |████████████████████████████████| 2.9MB 34.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=86e41ac9475345f64f1c51a9f0db390f11d821661608b0a588990e7a275a6b25
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.optimizers import Adam
import transformers
from transformers import TFAutoModelWithLMHead, AutoTokenizer
import logging
import json
# no extensive logging 
logging.getLogger().setLevel(logging.NOTSET)

AUTO = tf.data.experimental.AUTOTUNE

2.4.0


In [3]:
with open("sentences_dataset.json", "r" , encoding="utf-8") as fin:
  sentences = json.load(fin)

In [11]:
MAX_LEN = 128
BATCH_SIZE = 8 # per TPU core
TOTAL_STEPS = (len(sentences) // (BATCH_SIZE * 8) ) * 4 # thats approx 4 epochs
EVALUATE_EVERY = (len(sentences) // (BATCH_SIZE * 8) ) // 2
LR =  1e-5

PRETRAINED_MODEL = 'nlpaueb/bert-base-greek-uncased-v1'

In [12]:
def connect_to_TPU():
    """Detect hardware, return appropriate distribution strategy"""
    try:
        # TPU detection. No parameters necessary if TPU_NAME environment variable is
        # set: this is always the case on Kaggle.
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        tpu = None

    if tpu:
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.TPUStrategy(tpu)
    else:
        # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
        strategy = tf.distribute.get_strategy()

    global_batch_size = BATCH_SIZE * strategy.num_replicas_in_sync

    return tpu, strategy, global_batch_size


tpu, strategy, global_batch_size = connect_to_TPU()
print("REPLICAS: ", strategy.num_replicas_in_sync)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Running on TPU  grpc://10.4.164.194:8470
INFO:tensorflow:Initializing the TPU system: grpc://10.4.164.194:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.4.164.194:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [13]:
%%time

def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_mask=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen,
        truncation=True
    )
    
    return np.array(enc_di['input_ids'])
    

tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)
X_train = regular_encode(sentences, tokenizer, maxlen=MAX_LEN)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /nlpaueb/bert-base-greek-uncased-v1/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 140372454914872 on /root/.cache/huggingface/transformers/020dab34d496eb711abcd1a2fcb98999e8e52a647a19a30d587ecf0dced64b4d.3c398df2e83e354046a333f011627deb67111e36b02711ab77c2f4004a414da9.lock
INFO:filelock:Lock 140372454914872 acquired on /root/.cache/huggingface/transformers/020dab34d496eb711abcd1a2fcb98999e8e52a647a19a30d587ecf0dced64b4d.3c398df2e83e354046a333f011627deb67111e36b02711ab77c2f4004a414da9.lock
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /nlpaueb/bert-base-greek-uncased-v1/resolve/main/config.json HTTP/1.1" 200 459


DEBUG:filelock:Attempting to release lock 140372454914872 on /root/.cache/huggingface/transformers/020dab34d496eb711abcd1a2fcb98999e8e52a647a19a30d587ecf0dced64b4d.3c398df2e83e354046a333f011627deb67111e36b02711ab77c2f4004a414da9.lock
INFO:filelock:Lock 140372454914872 released on /root/.cache/huggingface/transformers/020dab34d496eb711abcd1a2fcb98999e8e52a647a19a30d587ecf0dced64b4d.3c398df2e83e354046a333f011627deb67111e36b02711ab77c2f4004a414da9.lock
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /nlpaueb/bert-base-greek-uncased-v1/resolve/main/vocab.txt HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 140372454914872 on /root/.cache/huggingface/transformers/36a03508d333d6195f8f456e8bcdebf6c61af8792f946a34b08a4ee5d7c0f86d.4157123280ff09762137030a9b558fcd76bd138c8e0275b9088621f0c807fbf5.lock
INFO:filelock:Lock 140372454914872 acquired on /root/.cache/huggingface/transformers/36a0350

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /nlpaueb/bert-base-greek-uncased-v1/resolve/main/vocab.txt HTTP/1.1" 200 529930


DEBUG:filelock:Attempting to release lock 140372454914872 on /root/.cache/huggingface/transformers/36a03508d333d6195f8f456e8bcdebf6c61af8792f946a34b08a4ee5d7c0f86d.4157123280ff09762137030a9b558fcd76bd138c8e0275b9088621f0c807fbf5.lock
INFO:filelock:Lock 140372454914872 released on /root/.cache/huggingface/transformers/36a03508d333d6195f8f456e8bcdebf6c61af8792f946a34b08a4ee5d7c0f86d.4157123280ff09762137030a9b558fcd76bd138c8e0275b9088621f0c807fbf5.lock
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /nlpaueb/bert-base-greek-uncased-v1/resolve/main/tokenizer.json HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /nlpaueb/bert-base-greek-uncased-v1/resolve/main/added_tokens.json HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /nlpaueb/bert-base-greek-uncased-v1/resolve/main/special_tokens_map.json HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 140372454914872 on /root/.cache/huggingface/transformers/347c648cda48ba9d519777e5e1e677a659862faeb5ad9e219ff0a95c62cfdea4.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d.lock
INFO:filelock:Lock 140372454914872 acquired on /root/.cache/huggingface/transformers/347c648cda48ba9d519777e5e1e677a659862faeb5ad9e219ff0a95c62cfdea4.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d.lock
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /nlpaueb/bert-base-gree

DEBUG:filelock:Attempting to release lock 140372454914872 on /root/.cache/huggingface/transformers/347c648cda48ba9d519777e5e1e677a659862faeb5ad9e219ff0a95c62cfdea4.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d.lock
INFO:filelock:Lock 140372454914872 released on /root/.cache/huggingface/transformers/347c648cda48ba9d519777e5e1e677a659862faeb5ad9e219ff0a95c62cfdea4.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d.lock
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /nlpaueb/bert-base-greek-uncased-v1/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 140372454914872 on /root/.cache/huggingface/transformers/8bc519ac7a17e04e2e2de1908d5c2670888449f1489da619329762a3b21f6eb9.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b.lock
INFO:filelock:Lock 140372454914872 acquired on /root/.cache/huggingface/transfor

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /nlpaueb/bert-base-greek-uncased-v1/resolve/main/tokenizer_config.json HTTP/1.1" 200 2


DEBUG:filelock:Attempting to release lock 140372454914872 on /root/.cache/huggingface/transformers/8bc519ac7a17e04e2e2de1908d5c2670888449f1489da619329762a3b21f6eb9.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b.lock
INFO:filelock:Lock 140372454914872 released on /root/.cache/huggingface/transformers/8bc519ac7a17e04e2e2de1908d5c2670888449f1489da619329762a3b21f6eb9.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b.lock
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,



CPU times: user 4.45 s, sys: 224 ms, total: 4.68 s
Wall time: 4.04 s


In [14]:
def prepare_mlm_input_and_labels(X):
    # 15% BERT masking
    inp_mask = np.random.rand(*X.shape)<0.15 
    # do not mask special tokens
    inp_mask[X<=103] = False
    # set targets to -1 by default, it means ignore
    labels =  -1 * np.ones(X.shape, dtype=int)
    # set labels for masked tokens
    labels[inp_mask] = X[inp_mask]
    
    # prepare input
    X_mlm = np.copy(X)
    # set input to [MASK] which is the last token for the 90% of tokens
    # this means leaving 10% unchanged
    inp_mask_2mask = inp_mask  & (np.random.rand(*X.shape)<0.90)
    X_mlm[inp_mask_2mask] = 103  # mask token is the last in the dict

    # set 10% to a random token
    inp_mask_2random = inp_mask_2mask  & (np.random.rand(*X.shape) < 1/9)
    X_mlm[inp_mask_2random] = np.random.randint(3, 250001, inp_mask_2random.sum())
    
    return X_mlm, labels



# masks and labels
X_train_mlm, y_train_mlm = prepare_mlm_input_and_labels(X_train)

In [15]:
def create_dist_dataset(X, y=None, training=False):
    dataset = tf.data.Dataset.from_tensor_slices(X)

    ### Add y if present ###
    if y is not None:
        dataset_y = tf.data.Dataset.from_tensor_slices(y)
        dataset = tf.data.Dataset.zip((dataset, dataset_y))
        
    ### Repeat if training ###
    if training:
        dataset = dataset.shuffle(len(X)).repeat()

    dataset = dataset.batch(global_batch_size).prefetch(AUTO)

    ### make it distributed  ###
    dist_dataset = strategy.experimental_distribute_dataset(dataset)

    return dist_dataset
    
    
train_dist_dataset = create_dist_dataset(X_train_mlm, y_train_mlm, True)

In [35]:
%%time

def create_mlm_model_and_optimizer():
    with strategy.scope():
        model = TFAutoModelWithLMHead.from_pretrained(PRETRAINED_MODEL)
        optimizer = tf.keras.optimizers.Adam(learning_rate=LR)
    return model, optimizer


mlm_model, optimizer = create_mlm_model_and_optimizer()
mlm_model.summary()

/usr/local/lib/python3.6/dist-packages/transformers/models/auto/modeling_tf_auto.py:776: FutureWarning: The class `TFAutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `TFAutoModelForCausalLM` for causal language models, `TFAutoModelForMaskedLM` for masked language models and `TFAutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /nlpaueb/bert-base-greek-uncased-v1/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /nlpaueb/bert-base-greek-uncased-v1/resolve/main/tf_model.h5 HTTP/1.1" 302 0
All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at nlpaueb/bert-b

Model: "tf_bert_for_masked_lm"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  112330752 
_________________________________________________________________
mlm___cls (TFBertMLMHead)    multiple                  27903416  
Total params: 112,957,880
Trainable params: 112,957,880
Non-trainable params: 0
_________________________________________________________________
CPU times: user 8.27 s, sys: 5.39 s, total: 13.7 s
Wall time: 43.9 s


In [17]:
def define_mlm_loss_and_metrics():
    with strategy.scope():
        mlm_loss_object = masked_sparse_categorical_crossentropy

        def compute_mlm_loss(labels, predictions):
            per_example_loss = mlm_loss_object(labels, predictions)
            loss = tf.nn.compute_average_loss(
                per_example_loss, global_batch_size = global_batch_size)
            return loss

        train_mlm_loss_metric = tf.keras.metrics.Mean()
        
    return compute_mlm_loss, train_mlm_loss_metric


def masked_sparse_categorical_crossentropy(y_true, y_pred):
    y_true_masked = tf.boolean_mask(y_true, tf.not_equal(y_true, -1))
    y_pred_masked = tf.boolean_mask(y_pred, tf.not_equal(y_true, -1))
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true_masked,
                                                          y_pred_masked,
                                                          from_logits=True)
    return loss

            
            
def train_mlm(train_dist_dataset, total_steps=2000, evaluate_every=200):
    step = 0
    ### Training lopp ###
    for tensor in train_dist_dataset:
        distributed_mlm_train_step(tensor) 
        step+=1

        if (step % evaluate_every == 0):   
            ### Print train metrics ###  
            train_metric = train_mlm_loss_metric.result().numpy()
            print("Step %d, train loss: %.2f" % (step, train_metric))     

            ### Reset  metrics ###
            train_mlm_loss_metric.reset_states()
            
        if step  == total_steps:
            break


@tf.function
def distributed_mlm_train_step(data):
    strategy.run(mlm_train_step, args=(data,))


@tf.function
def mlm_train_step(inputs):
    features, labels = inputs

    with tf.GradientTape() as tape:
        predictions = mlm_model(features, training=True)[0]
        loss = compute_mlm_loss(labels, predictions)

    gradients = tape.gradient(loss, mlm_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, mlm_model.trainable_variables))

    train_mlm_loss_metric.update_state(loss)
    

compute_mlm_loss, train_mlm_loss_metric = define_mlm_loss_and_metrics()

In [18]:
train_mlm(train_dist_dataset, TOTAL_STEPS, EVALUATE_EVERY)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7fabab058e58> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).WARNING:tensorflow:AutoGraph could not transform <bound method Socket.send of <zmq.sugar.socket.Socket object at 0x7fab9380f660>> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7fabab058e58> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7fabab058e58> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Step 115, train loss: 1.25
Step 230, train loss: 1.13
Step 345, train loss: 1.00
Step 460, train loss: 1.00
Step 575, train loss: 0.88
Step 690, train loss: 0.89
Step 805, train loss: 0.77
Step 920, train loss: 0.79


In [34]:
mlm_model.save_pretrained('greekBERT')

In [37]:
text_1 = '''στην επιδημιολογία του [MASK] του τραχήλου της [MASK] συμπεριλαμβάνεται και η [MASK] 
από τον ιό των ανθρώπινων θηλωμάτων ([MASK]).'''
text_1 = '''Το τσιγάρο κάνει [MASK] στην υγεία και προκαλεί [MASK] '''
# text_1 = 'η ραβδομυόλυση ορίζεται ως [MASK], το οποίο εμφανίζεται μετά από καταστροφή (λύση) ή διαταραχή στην παραγωγή/κατανάλωση ενέργειας των μυϊκών κυττάρων'
# EN: 'The poet wrote a [MASK].'
input_ids = tokenizer.encode(text_1)
print(tokenizer.convert_ids_to_tokens(input_ids))
len(input_ids)

['[CLS]', 'το', 'τσιγαρο', 'κανει', '[MASK]', 'στην', 'υγεια', 'και', 'προκαλει', '[MASK]', '[SEP]']


11

In [39]:
input_ids = tokenizer.encode(text_1, return_tensors="tf")
outputs = mlm_model(input_ids)
output_ids = []
for i in range(0,11):
  output_ids.append(tf.math.argmax(outputs.logits[0][i]).numpy())

print(tokenizer.decode(output_ids))
print(output_ids)

) το τσιγαρο κανει κακο στην υγεια και προκαλει καρκινο.
[116, 345, 9148, 451, 1190, 360, 1045, 344, 1870, 2997, 121]
